In [1]:
import os
import subprocess
from calendar import monthrange
import pandas as pd

In [2]:
os.environ["SLURM_TIME_FORMAT"] = "%s"

In [3]:
def get_data_from_sacct(clusters: str,
                        start_date: str,
                        end_date: str,
                        users: str,
                        fields: str) -> pd.DataFrame:
    """Return a dataframe of the sacct output."""
    cmd = f"sacct -M {clusters} -X -P -n -S {start_date} -E {end_date} -u {users} -o {fields}"
    output = subprocess.run(cmd,
                            stdout=subprocess.PIPE,
                            shell=True,
                            timeout=100,
                            text=True,
                            check=True)
    rows = [row.split("|") for row in output.stdout.split()]
    df = pd.DataFrame(rows)
    df.columns = fields.split(",")
    return df

In [4]:
def clean_dataframe(df):
    for col in ["elapsedraw", "start", "eligible"]:
        df = df[pd.notna(df[col])]
        df = df[df[col].str.isnumeric()]
        df[col] = df[col].astype("int64")
    df = df[df.elapsedraw > 0]
    df.cputimeraw = df.cputimeraw.astype("int64")
    return df

In [5]:
users = "yp1065,yongickc,fanc,jj2075,hw1538,zl8026,sc2097,ss0410,wc1202,mk3254,ak9088,tianhaol"
fields = "user,cputimeraw,elapsedraw,start,eligible"

In [6]:
print("start", "end", "Q-hours", "CPU-hours")
for i in range(1, 8):
    end_day = monthrange(2024, i)[1]
    s = f"2024-0{i}-01"
    e = f"2024-0{i}-{end_day}"
    df = get_data_from_sacct("della", s, e, users, fields)
    df = clean_dataframe(df)
    df["q-seconds"] = df.start - df.eligible
    print(s, e, round(df["q-seconds"].sum() / 3600), round(df["cputimeraw"].sum() / 3600))
    #print(df.groupby("user").agg({"q-seconds":sum}))

start end Q-hours CPU-hours
2024-01-01 2024-01-31 13077 122529
2024-02-01 2024-02-29 29504 208377
2024-03-01 2024-03-31 92142 297380
2024-04-01 2024-04-30 137545 317626
2024-05-01 2024-05-31 55810 376075
2024-06-01 2024-06-30 44560 259683
2024-07-01 2024-07-31 90770 390156
